In [1]:
import json
import sqlite3
import pandas as pd

## Read Excel, make `coords` column for collision lookup

Workshops times (*horarios*) are modeled as a categoricall int tuple `( <day time>, <week day> )`:

- Day times
    - `0` first workshops block
    - `1` second workshops block
- Week days
    - `0` First week day "*Lunes*"
    - `1` Second week day "*Martes*"
    - ...


In [3]:
def ws_table(drop_c = False):
    t = pd.read_excel('talleres.ods', sheet_name='talleres')
    t['coords'] = t.loc[:,'c1':'c3'].apply(row_coords, axis=1)
    if drop_c:
        t = t.drop(columns = t.loc[:,'c1':'c3'].columns)
    return t

def tup_from_cell_coord(s):
    assert isinstance(s,str)
    day_idx = {
        'lu': 0,
        'ma': 1,
        'mi': 2,
        'ju': 3,
        'vi': 4,
           }
    d, h = s[:2], s[2:]
    return (int(h)-1, day_idx[d.lower()])

def row_coords(row):
    r = row[row.notna()]
    return r.apply(tup_from_cell_coord).to_list()

t = ws_table(drop_c = True)
t.sample(10)

,name,teacher,cycle,full_name,description,coords
23,Encuadernación,Javiera,canelos y manios,NaN,NaN,"[(0, 4)]"
44,Inglés B,Francisca U.,coihues y avellanos,NaN,NaN,"[(1, 4)]"
50,Gimnasia artística,Paula,canelos y manios,NaN,NaN,"[(1, 4)]"
7,Comics,Grace,canelos y manios,NaN,NaN,"[(0, 2), (1, 4)]"
36,Percusión corporal,Marcelo,canelos y manios,NaN,NaN,"[(1, 3)]"
18,Vectores,Salvador,coihues y avellanos,NaN,NaN,"[(0, 0)]"
24,Cultura y países I,Javiera,coihues y avellanos,Historia de culturas y países,NaN,"[(1, 0)]"
16,Fractángulos,Salvador,canelos y manios,NaN,NaN,"[(0, 1)]"
42,Artes,Francisca U.,canelos y manios,NaN,NaN,"[(0, 4)]"
12,Jardineria y cultivo,Cristina,canelos y manios,Jardineria y cultivo de plantas,NaN,"[(0, 0), (0, 3)]"


## Collision of workshop coordinates

Make a boolean dataframe that test for coincidence in workshops `coords` list.

`collision_df(series)` returns the boolean coincidence table beetwen every workshop pair coordinates pair. 

In [13]:
def sample_collision(sample, df):
    return (df.astype("string") == str(sample)).any(axis=1)
    
def row_collision(row, df):
    bol = row.apply(sample_collision, df=df).any()
    return bol #df.loc[bol].index

def collision_df(series):
    df = series.apply(pd.Series)
    return df.apply(row_collision, df=df, axis=1)

t = ws_table(drop_c = True)
#t = t[t.cycle=='ulmos']
collision_df(t.coords)

,0,1,2,3,4,5,6,7,8,9,...,43,44,45,46,47,48,49,50,51,52
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,False,True,False,False,False,False,False,True,True,False,...,False,True,False,False,False,True,False,True,False,False
2,False,False,True,False,False,False,False,False,False,True,...,False,False,True,False,False,False,False,False,False,True
3,False,False,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4,False,False,False,False,True,False,False,False,True,False,...,False,False,False,True,False,False,False,False,False,False
5,False,False,False,False,False,True,False,True,False,False,...,True,False,False,False,True,False,True,False,False,False
6,False,False,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
7,False,True,False,False,False,True,False,True,False,False,...,False,True,False,False,False,True,False,True,False,False
8,False,True,False,False,True,False,False,False,True,False,...,False,False,False,True,False,False,False,False,False,False
9,False,False,True,False,False,False,False,False,False,True,...,False,False,True,False,False,False,False,False,False,True


In [14]:
t.loc[[52,2]]

,name,teacher,cycle,full_name,description,coords
52,Didáctica matemática,Paula,canelos y manios,NaN,NaN,"[(1, 0)]"
2,Pintura I,Carla,canelos y manios,NaN,NaN,"[(1, 0)]"


## Json serialize

In [18]:
t = ws_table(drop_c = True)
t = t[t.cycle == 'canelos y manios']
json_str = collision_df(t.coords).to_json()
print(f'{json_str[:500]} ...')

{"2":{"2":true,"3":false,"4":false,"7":false,"12":false,"13":false,"14":false,"16":false,"17":false,"22":false,"23":false,"27":false,"35":false,"36":false,"41":false,"42":false,"49":false,"50":false,"52":true},"3":{"2":false,"3":true,"4":false,"7":false,"12":true,"13":false,"14":false,"16":false,"17":true,"22":false,"23":false,"27":false,"35":false,"36":false,"41":false,"42":false,"49":false,"50":false,"52":false},"4":{"2":false,"3":false,"4":true,"7":false,"12":false,"13":false,"14":false,"16": ...


## Label format for input switches view

In [47]:
t = ws_table(drop_c = True)
t = t[t.cycle == 'canelos y manios']
s = t.loc[20]
print(f'{s["name"]} con {s.teacher}, horarios: {s.coords}')
s

Encuadernación con Javiera, horarios: [(0, 4)]


name             Encuadernación
teacher                 Javiera
cycle          canelos y manios
full_name                   NaN
description                 NaN
coords                 [(0, 4)]
Name: 20, dtype: object

## Sort workshops by week day and time

In [83]:
def sort_by_day_time(t):
    df = pd.DataFrame(t.coords.str[0].to_list(), index=t.index)
    idx = df.sort_values([1,0]).index
    return t.loc[idx]

t = ws_table(drop_c = True)
t = t[t.cycle == 'canelos y manios']
t = sort_by_day_time(t)
t

,name,teacher,cycle,full_name,description,coords
10,Jardineria y cultivo,Cristina,canelos y manios,Jardineria y cultivo de plantas,NaN,"[(0, 0), (0, 3)]"
23,Explorando la naturaleza,Alejandro,canelos y manios,Historia de la pintura y filosofía,NaN,"[(0, 0)]"
29,Filosofía,Marcelo,canelos y manios,NaN,NaN,"[(0, 0), (1, 3)]"
2,Color y expresión,Carla,canelos y manios,NaN,NaN,"[(1, 0), (0, 3)]"
38,Didáctica matemática,Paula,canelos y manios,NaN,NaN,"[(1, 0)]"
13,Fractángulos,Salvador,canelos y manios,NaN,NaN,"[(0, 1)]"
19,Ave y fauna,Javiera,canelos y manios,NaN,NaN,"[(0, 1)]"
32,Inglés,Francisca U.,canelos y manios,NaN,NaN,"[(0, 1), (1, 3)]"
11,Cocina,Cristina,canelos y manios,NaN,NaN,"[(1, 1), (0, 2)]"
37,Juegos matemáticos,Paula,canelos y manios,NaN,NaN,"[(1, 1), (1, 4)]"


## Group workshops by coords for input switches view

Need to group workshops by day and time in order to present within same title in the web input switches selection view. The first coordinate of every workshop for certain cycle is used for generating an input switch. The remaining coordinates of every workshop are used to write a small text within the corresponding title but without a switch (every workshop is selected only in it first day and time).

In [106]:
t = ws_table(drop_c = True)
t = sort_by_day_time(t)
g = t.groupby([t.coords.str[0].str[1], t.coords.str[0].str[0]]).groups
{k: [f'ws_{i}' for i in v] for k,v in g.items()}

{(0, 0): ['ws_0',
  'ws_10',
  'ws_15',
  'ws_17',
  'ws_23',
  'ws_29',
  'ws_30',
  'ws_39'],
 (0, 1): ['ws_2', 'ws_8', 'ws_21', 'ws_27', 'ws_35', 'ws_38'],
 (1, 0): ['ws_1', 'ws_7', 'ws_13', 'ws_19', 'ws_22', 'ws_32'],
 (1, 1): ['ws_4', 'ws_11', 'ws_26', 'ws_34', 'ws_36', 'ws_37'],
 (2, 0): ['ws_6', 'ws_24', 'ws_31'],
 (2, 1): ['ws_3', 'ws_9', 'ws_12', 'ws_28'],
 (3, 0): ['ws_5', 'ws_14', 'ws_18'],
 (3, 1): ['ws_25'],
 (4, 0): ['ws_16', 'ws_20', 'ws_33']}

## Week schedule list for week schedule view

Use `week_schedule(d)` to make a complete week schedule dict from a partial dict `d`. Use `week_table_rows(d)` for output `d` dict values in row list ready to use in a table template.

In [87]:
days_list = ['Lu', 'Ma', 'Mi', 'Ju', 'Vi']
times_list = [('10:15', '11:15'), ('12:30', '13:30')]
parallels = 3

workshop_times = {
    (2,1): 'Cristina',
    (2,3): 'Cristina',
    (1,2): 'Grace',
    (2,2): 'Carla',
    (3,2): 'Marcelo',
    (3,5): 'Marcelo',
    (2,4): 'Paula',
    (1,4): 'Fran'
}

def week_schedule(d):
    new = {(a,b): '' for a in range(1,4) for b in range(1,6)}
    return {**new, **d}

def week_table_rows(d):
    headers = [
        ('10:00', '11:00'),
        ('11:15', '12:15'),
        ('12:30', '13:30')
    ]
    return [[d[(a,b)] for b in range(1,6)] for a in range(1,4)]

d = week_schedule(workshop_times)
for a,b in zip(['10','11','12'],week_table_rows(d)):
    print(a, b)

10 ['', 'Grace', '', 'Fran', '']
11 ['Cristina', 'Carla', 'Cristina', 'Paula', '']
12 ['', 'Marcelo', '', '', 'Marcelo']


In [100]:
workshop_times = {(a,b): [] for a in range(len(times_list))
                         for b in range(len(days_list))}
t = ws_table(drop_c = True)
post = {10: True, 23: False, 29: True, 2: False, 38: True, 13: True, 19: True, 32: False, 11: True, 37: False, 6: False, 3: True, 28: False, 14: False, 20: False, 33: False}
idx = [i for i in post if post[i]]

for i in post:
    if post[i]:
        for coord in t.loc[i,'coords']:
            workshop_times[coord].append(t.loc[i,'name'])

{k: (v[0] if len(v)>0 else '')
 for k, v in workshop_times.items()
 }

{(0, 0): 'Jardineria y cultivo',
 (0, 1): 'Fractángulos',
 (0, 2): 'Cocina',
 (0, 3): 'Jardineria y cultivo',
 (0, 4): '',
 (1, 0): 'Didáctica matemática',
 (1, 1): 'Cocina',
 (1, 2): 'Títeres',
 (1, 3): 'Filosofía',
 (1, 4): ''}

In [21]:
DAYS = ['Lu', 'Ma', 'Mi', 'Ju', 'Vi']
TIMES = [('10:15', '11:15'), ('12:30', '13:30')]
PARALLELS = 3
    
options_dict = {
    (2,1): ['Cristina', 'Grace'],
    (2,3): ['Cristina', 'Carla'],
    (1,2): ['Jano', 'Carla'],
    (2,2): ['Fran', 'Jano'],
    (1,1): ['Marcelo', 'Cristina'],
    (1,5): ['Paula', 'Carla', 'Jano'],
    (2,4): ['Pauli', 'Grace'],
    (1,4): ['Grace', 'Javi']
}

def html_field_name(tup):
    return f"cell_{'_'.join([str(c) for c in tup])}"

def key_tuple(name):
    _,a,b = name.split('_')
    return int(a), int(b)

key_tuple('cell_2_1')

(2, 1)

In [22]:
html_field_name((2,1))

'cell_2_1'

## Sqlite DataBase

Python sqlite3 tutorial with some real data subset.

In [2]:
#This create the file inmediatly if doesn't exist
con = sqlite3.connect('wshop.db')

In [3]:
cur = con.cursor()

In [4]:
cur.execute("CREATE TABLE taller(nombre, profe, ciclo)")

In [5]:
res = cur.execute("SELECT name from sqlite_master")
res.fetchone()

('taller',)

In [13]:
cur.execute("""
    INSERT INTO taller VALUES
        ('Tejido', 'Javiera', 'ulmos'),
        ('Reciclaje de plásticos','Cristina', 'ulmos'),
        ('Artes', 'Francisca U.', 'ulmos')
""")

In [14]:
con.commit()

In [15]:
res = cur.execute("SELECT nombre FROM taller")
res.fetchall()

[('Tejido',), ('Reciclaje de plásticos',), ('Artes',)]

In [24]:
t = ws_table(drop_c = True)
data = []
for i in range(10,20):
    row = tuple([
        t.loc[i,col] for col in ['name','teacher','cycle']])
    data.append(row)

cur.executemany("INSERT INTO taller VALUES(?,?,?)",
                data)
con.commit()
#t.loc[10:20,['name','teacher','cycle']].to_list()
# s = t.loc[40]
# print(f'{s["name"]} con {s.teacher}, horarios: {s.coords}')
# s

In [25]:
for row in cur.execute(
    "SELECT nombre, profe, ciclo FROM taller ORDER BY ciclo"):
    print(row)

('Jardineria y cultivo', 'Cristina', 'canelos y manios')
('Cocina', 'Cristina', 'canelos y manios')
('Reciclaje de plásticos', 'Cristina', 'canelos y manios')
('Fractángulos', 'Salvador', 'canelos y manios')
('TecnoSapiens', 'Salvador', 'canelos y manios')
('Vectores', 'Salvador', 'coihues y avellanos')
('Nutricina', 'Salvador', 'coihues y avellanos')
('Tejido', 'Javiera', 'ulmos')
('Reciclaje de plásticos', 'Cristina', 'ulmos')
('Artes', 'Francisca U.', 'ulmos')
('Reciclaje de plásticos', 'Cristina', 'ulmos')
('Cocina', 'Cristina', 'ulmos')
('Secuencias curiosas', 'Salvador', 'ulmos')


In [26]:
con.close()
new_con = sqlite3.connect("wshop.db")
new_cur = new_con.cursor()
res = new_cur.execute(
    "SELECT nombre, profe, ciclo FROM taller")
for row in res:
    print(row)

('Tejido', 'Javiera', 'ulmos')
('Reciclaje de plásticos', 'Cristina', 'ulmos')
('Artes', 'Francisca U.', 'ulmos')
('Reciclaje de plásticos', 'Cristina', 'ulmos')
('Cocina', 'Cristina', 'ulmos')
('Jardineria y cultivo', 'Cristina', 'canelos y manios')
('Cocina', 'Cristina', 'canelos y manios')
('Reciclaje de plásticos', 'Cristina', 'canelos y manios')
('Secuencias curiosas', 'Salvador', 'ulmos')
('Fractángulos', 'Salvador', 'canelos y manios')
('TecnoSapiens', 'Salvador', 'canelos y manios')
('Vectores', 'Salvador', 'coihues y avellanos')
('Nutricina', 'Salvador', 'coihues y avellanos')
